In [20]:
# todo 学習結果を使って運用するための環境を作る。
# 選手情報・過去レース情報から3連単舟券120種をクラス分類する
# LGBMでやってみる。
# todo:bayseの方と、インプットの与え方をあわせる。今はOne-Hotが一致していない。
# 的中予想確率10%以上のうち、オッズで見合うものだけ買った場合のシミュレーションをする。
# このnoteでは情報を増やしてみる。

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import csv

In [21]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler


In [22]:
# 舟券の配列を取得
funakenList=[]
with open(os.environ['BR_HOME']+'/boatrace/config/3t_list.dat') as f:
    reader=csv.reader(f)
    for row in reader:
        funakenList.append(row)
funakenID = [i for i in range(120)]
funakenDict=dict(zip(funakenList[0],funakenID))
print(funakenDict)

{'1-2-3': 0, '1-2-4': 1, '1-2-5': 2, '1-2-6': 3, '1-3-2': 4, '1-3-4': 5, '1-3-5': 6, '1-3-6': 7, '1-4-2': 8, '1-4-3': 9, '1-4-5': 10, '1-4-6': 11, '1-5-2': 12, '1-5-3': 13, '1-5-4': 14, '1-5-6': 15, '1-6-2': 16, '1-6-3': 17, '1-6-4': 18, '1-6-5': 19, '2-1-3': 20, '2-1-4': 21, '2-1-5': 22, '2-1-6': 23, '2-3-1': 24, '2-3-4': 25, '2-3-5': 26, '2-3-6': 27, '2-4-1': 28, '2-4-3': 29, '2-4-5': 30, '2-4-6': 31, '2-5-1': 32, '2-5-3': 33, '2-5-4': 34, '2-5-6': 35, '2-6-1': 36, '2-6-3': 37, '2-6-4': 38, '2-6-5': 39, '3-1-2': 40, '3-1-4': 41, '3-1-5': 42, '3-1-6': 43, '3-2-1': 44, '3-2-4': 45, '3-2-5': 46, '3-2-6': 47, '3-4-1': 48, '3-4-2': 49, '3-4-5': 50, '3-4-6': 51, '3-5-1': 52, '3-5-2': 53, '3-5-4': 54, '3-5-6': 55, '3-6-1': 56, '3-6-2': 57, '3-6-4': 58, '3-6-5': 59, '4-1-2': 60, '4-1-3': 61, '4-1-5': 62, '4-1-6': 63, '4-2-1': 64, '4-2-3': 65, '4-2-5': 66, '4-2-6': 67, '4-3-1': 68, '4-3-2': 69, '4-3-5': 70, '4-3-6': 71, '4-5-1': 72, '4-5-2': 73, '4-5-3': 74, '4-5-6': 75, '4-6-1': 76, '4-6-2':

In [23]:
# 分析期間の指定は一旦ここでまとめてみる。
trainStartDate="20170401"
trainEndDate="20190430"
# test はtrainからsplitする

In [24]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [25]:
# trainの元データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (trainStartDate,trainEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("traindata:",len(loadList))

traindata: 113095


In [26]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,...,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,1-5-2,0,0.3333,0.4848,0.3817,0.5645,B1,4.842381,0,0.3333,...,0.4452,0.3134,0.4478,B2,0.594831,30.3,2017-04-01,20170401-02-01,1,2.0
1,2-1-3,1,0.2876,0.4248,0.3960,0.6040,B1,7.406036,0,0.3871,...,0.4177,0.4500,0.5750,B1,1.504087,79.0,2017-04-01,20170401-02-02,1,2.0
2,2-3-1,1,0.2452,0.4387,0.4610,0.5779,B1,8.340631,1,0.3807,...,0.4938,0.3541,0.5502,A2,3.195143,28.9,2017-04-01,20170401-02-03,1,3.0
3,1-4-2,0,0.1959,0.3986,0.6369,0.7738,B1,9.654840,0,0.3439,...,0.3846,0.3161,0.4540,B1,0.583962,7.7,2017-04-01,20170401-02-04,1,3.0
4,1-2-4,0,0.3609,0.4852,0.2983,0.4475,A2,15.248551,0,0.2911,...,0.5375,0.4296,0.6667,A2,2.543009,52.7,2017-04-01,20170401-02-05,1,2.0


In [27]:
# 入力のデータ整形
#xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
#xdf.head()

# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
#xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
rankLabel=LabelEncoder()
rankLabel=rankLabel.fit(xdf['l1rank'])
xdf['l1rank']=rankLabel.transform(xdf['l1rank'])
xdf['l2rank']=rankLabel.transform(xdf['l2rank'])
xdf['l3rank']=rankLabel.transform(xdf['l3rank'])
xdf['l4rank']=rankLabel.transform(xdf['l4rank'])
xdf['l5rank']=rankLabel.transform(xdf['l5rank'])
xdf['l6rank']=rankLabel.transform(xdf['l6rank'])
xdf.head()


,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,l2boat3r,...,l5score,l6Fcnt,l6boat2r,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,raceWaveHeight,raceWindSpeed
0,0,0.3333,0.4848,0.3817,0.5645,2,4.842381,0,0.3333,0.5094,...,2.860248,0,0.2740,0.4452,0.3134,0.4478,3,0.594831,1,2.0
1,1,0.2876,0.4248,0.3960,0.6040,2,7.406036,0,0.3871,0.5935,...,2.015905,1,0.2532,0.4177,0.4500,0.5750,2,1.504087,1,2.0
2,1,0.2452,0.4387,0.4610,0.5779,2,8.340631,1,0.3807,0.5739,...,1.279918,0,0.3125,0.4938,0.3541,0.5502,1,3.195143,1,3.0
3,0,0.1959,0.3986,0.6369,0.7738,2,9.654840,0,0.3439,0.5350,...,1.545834,0,0.2231,0.3846,0.3161,0.4540,2,0.583962,1,3.0
4,0,0.3609,0.4852,0.2983,0.4475,1,15.248551,0,0.2911,0.4557,...,11.021105,1,0.3500,0.5375,0.4296,0.6667,1,2.543009,1,2.0


In [28]:
# ファイルから作った辞書で変換する
ydf=df['funaken']
ydf=pd.DataFrame(ydf.replace(funakenDict))
print(ydf.head())


   funaken
0       12
1       20
2       24
3        8
4        1


In [29]:
# 重み付けのため、オッズのリストを作る
odf=df['odds'].values
#odf=pd.DataFrame(df['odds'])
#odf.describe()
print(odf)

[30.3 79.  28.9 ... 15.  21.7 10.6]


In [50]:
X_train, X_test, y_train, y_test,o_train,o_test = train_test_split(xdf, ydf,odf)
print("X_train,X_test:",len(X_train),len(X_test))

X_train,X_test: 84821 28274


In [51]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [52]:
lgbm_params = {
    # 多値分類問題
    'objective': 'multiclass',
    # クラス数は 120
    'num_class': 120,
    #'class_weight':'balanced',
    'random_state':999,
    # 以下、ハイパーパラメタ
    'max_depth':3,
    'num_leaves':6,
    'min_data_in_leaf':300,
    # 正則化
    'reg_alpha':2.425,
    'reg_lambda':9.473 ,
}


In [53]:
lgb.LGBMClassifier()

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [54]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

[1]	valid_0's multi_logloss: 4.66107
[2]	valid_0's multi_logloss: 4.57071
[3]	valid_0's multi_logloss: 4.49984
[4]	valid_0's multi_logloss: 4.44113
[5]	valid_0's multi_logloss: 4.39107
[6]	valid_0's multi_logloss: 4.34738
[7]	valid_0's multi_logloss: 4.30879
[8]	valid_0's multi_logloss: 4.27512
[9]	valid_0's multi_logloss: 4.24436
[10]	valid_0's multi_logloss: 4.21695
[11]	valid_0's multi_logloss: 4.1925
[12]	valid_0's multi_logloss: 4.17028
[13]	valid_0's multi_logloss: 4.14996
[14]	valid_0's multi_logloss: 4.13138
[15]	valid_0's multi_logloss: 4.11409
[16]	valid_0's multi_logloss: 4.09813
[17]	valid_0's multi_logloss: 4.08359
[18]	valid_0's multi_logloss: 4.06995
[19]	valid_0's multi_logloss: 4.0573
[20]	valid_0's multi_logloss: 4.04572
[21]	valid_0's multi_logloss: 4.03504
[22]	valid_0's multi_logloss: 4.02479
[23]	valid_0's multi_logloss: 4.01553
[24]	valid_0's multi_logloss: 4.0067
[25]	valid_0's multi_logloss: 3.9985
[26]	valid_0's multi_logloss: 3.9908
[27]	valid_0's multi_loglo

In [55]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

# 精度 (Accuracy) を計算する
accuracy = sum(y_test['funaken'] == y_pred_max) / len(y_test)
print("accuracy:",accuracy)

# 回収率を計算
res=0
for i in range(len(y_test)):
    if (y_test.iloc[i])['funaken']==y_pred_max[i]:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        res += o_test[i] -1
    else:
        #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
        pass
print("resultReturn:",res/len(y_test))


accuracy: 0.08934002970927354
resultReturn: 0.710076395274811


In [56]:
print(len(y_pred))
# オッズを見て判断する場合
resAmount=0
buyAmount=0

dfIndexList=y_test['funaken'].index
for i in range(len(y_test)):
    raceId = df.iloc[dfIndexList[i]]['raceId']
    #print(raceId)
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)
        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
    #print((loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] )
    #if ( (loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] > 1) :
    #    print("ok")
    #print(y_pred[i][0])
    #print(raceId)
    for j in range(120):
        if y_pred[i][j]> 0.10 and ( (y_pred[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0])
            buyAmount+=1
            if y_test['funaken'].iloc[i]==j:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
            else:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                pass
print("resultReturn:",resAmount/buyAmount)


28274
buy: 20190103-17-08 13 17 16.1
buy: 20190320-18-02 85 0 13.5
☆hit!☆: 20190320-18-02 0 13.5
buy: 20180705-24-01 121 16 13.8
buy: 20190120-19-08 188 4 17.4
buy: 20190428-16-12 528 0 16.5
buy: 20170924-12-12 597 1 13.7
buy: 20180905-10-03 603 4 21.9
buy: 20190429-11-01 660 0 33.8
buy: 20190424-09-05 683 0 10.1
buy: 20170922-17-05 702 1 16.4
buy: 20171113-12-07 748 4 21.0
buy: 20180830-21-04 804 0 8.9
buy: 20190106-21-01 894 2 22.3
☆hit!☆: 20190106-21-01 2 22.3
buy: 20190106-21-01 894 12 15.9
buy: 20170622-18-04 898 0 14.5
buy: 20190125-14-05 924 0 14.3
buy: 20171210-09-01 995 1 10.9
buy: 20181225-10-10 999 12 16.7
☆hit!☆: 20181225-10-10 12 16.7
buy: 20181107-16-10 1009 0 14.6
buy: 20170725-09-05 1017 1 17.5
buy: 20180507-22-05 1122 0 17.2
buy: 20171027-21-01 1250 8 17.5
buy: 20181224-20-03 1280 4 15.2
buy: 20180406-09-04 1338 1 16.6
buy: 20180319-09-05 1349 0 12.9
buy: 20171117-08-08 1377 4 14.8
buy: 20180221-24-03 1383 5 19.4
buy: 20180320-01-06 1393 0 13.3
buy: 20190211-21-09 1400

buy: 20190205-10-04 10863 1 13.1
buy: 20180719-18-01 10870 0 14.4
buy: 20181111-13-05 10926 5 10.0
buy: 20190310-24-05 10979 4 22.6
buy: 20181212-17-09 10990 0 14.6
buy: 20181228-12-08 11049 0 15.8
buy: 20181225-22-04 11098 0 15.4
buy: 20180727-20-05 11116 0 19.1
buy: 20190303-10-03 11160 0 11.0
buy: 20170522-24-05 11170 4 9.5
buy: 20170620-15-06 11246 0 17.1
buy: 20170523-24-08 11280 0 18.8
buy: 20180906-17-06 11395 1 16.9
buy: 20180223-10-01 11401 1 14.6
buy: 20180428-12-08 11419 8 21.0
buy: 20180113-02-10 11465 0 14.0
☆hit!☆: 20180113-02-10 0 14.0
buy: 20170925-22-01 11498 8 13.6
buy: 20171214-24-03 11515 7 14.2
buy: 20180225-16-05 11595 9 16.8
buy: 20190113-22-04 11601 0 20.2
buy: 20170508-01-06 11640 1 10.8
buy: 20190329-18-10 11687 4 14.9
buy: 20181125-18-02 11761 0 14.0
buy: 20171128-13-05 11813 1 14.0
buy: 20171024-08-08 11863 1 16.9
buy: 20190407-07-01 11959 0 15.8
☆hit!☆: 20190407-07-01 0 15.8
buy: 20190407-07-01 11959 4 16.2
buy: 20190427-18-03 11987 0 19.4
buy: 20190205-23-

buy: 20180418-21-11 21567 4 10.1
buy: 20181126-07-08 21605 6 15.2
☆hit!☆: 20181126-07-08 6 15.2
buy: 20180328-10-01 21641 0 16.0
buy: 20190210-24-09 21702 3 16.9
buy: 20171129-24-03 21811 41 16.0
buy: 20180806-09-01 21939 0 26.8
buy: 20180806-09-01 21939 1 22.7
buy: 20171009-05-02 21948 20 22.5
buy: 20171002-20-04 21949 4 16.7
buy: 20190430-13-11 21972 0 15.6
☆hit!☆: 20190430-13-11 0 15.6
buy: 20180501-17-09 22032 0 13.8
buy: 20170420-21-01 22102 0 22.3
buy: 20180509-05-02 22140 0 12.0
buy: 20180709-22-08 22172 0 15.1
buy: 20171203-10-09 22209 23 20.3
buy: 20180118-17-04 22282 42 20.0
buy: 20181024-21-06 22321 20 13.3
buy: 20190312-13-07 22382 41 17.9
☆hit!☆: 20190312-13-07 41 17.9
buy: 20170728-23-04 22462 18 13.3
buy: 20171123-12-07 22471 4 17.8
buy: 20180829-04-09 22545 0 9.5
buy: 20181231-15-11 22552 0 9.4
buy: 20170603-15-08 22555 0 11.9
☆hit!☆: 20170603-15-08 0 11.9
buy: 20190114-13-02 22563 22 22.8
buy: 20180906-11-06 22619 0 20.9
☆hit!☆: 20180906-11-06 0 20.9
buy: 20180401-24-0

In [57]:
print("totalRace,buy,return",len(y_test),buyAmount,resAmount )
# 現状でも100%はロバストに超えそう。
# だがrrが1.2でもbuyが6/300レースだと、1000円投票しても一日だと約3000*1.2⇒利益600円なので、もっと回収率を上げるか母数増やす。
# totalRace,buy,return 28274 608 614.3
# totalRace,buy,return 28274 614 804.8


totalRace,buy,return 28274 614 804.8000000000002


In [58]:
print(len(y_pred))
# オッズを見て判断する場合
resAmount=0
buyAmount=0

dfIndexList=y_test['funaken'].index
for i in range(len(y_test)):
    raceId = df.iloc[dfIndexList[i]]['raceId']
    #print(raceId)
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)
        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
    #print((loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] )
    #if ( (loadList[loadList['funaken']==0]['odds'] * y_pred[i][0] ).values[0] > 1) :
    #    print("ok")
    #print(y_pred[i][0])
    #print(raceId)
    for j in range(120):
        # y_predの閾値を下げてみる。
        if y_pred[i][j]> 0.05 and ( (y_pred[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0])
            buyAmount+=1
            if y_test['funaken'].iloc[i]==j:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
            else:
            #print("i:",i,"result:",y_test[i],"forecast:",y_pred_max[i],"forecastProb:",y_pred[i][y_pred_max[i]],"return:",o_test[i],"expect:",y_pred[i][y_pred_max[i]]*o_test[i])
                pass
print("resultReturn:",resAmount/buyAmount)
print("totalRace,buy,return",len(y_test),buyAmount,resAmount )
# y_pred[i][j]> 0.05に閾値下げてみた。投票数はかなり増えているが、回収率が低く、トータルリターンは下がっている。
# ToDo：運用に向けてはこのバランスを調整してみるのはあり。
#resultReturn: 1.005279187817259
#totalRace,buy,return 28274 4925 4951.0

28274
buy: 20171030-18-08 12 16 15.3
buy: 20190103-17-08 13 17 16.1
buy: 20190412-11-08 18 1 31.8
buy: 20190412-11-08 18 5 29.8
buy: 20170504-11-03 21 37 19.6
buy: 20180123-17-05 23 2 20.7
buy: 20180712-14-06 24 0 22.0
buy: 20180712-14-06 24 4 27.5
buy: 20180227-09-09 27 1 27.7
buy: 20180227-09-09 27 5 33.5
buy: 20180730-17-06 28 0 23.1
buy: 20180730-17-06 28 20 26.0
buy: 20170428-23-01 32 0 16.6
buy: 20190320-18-05 39 1 19.5
buy: 20190320-18-05 39 11 60.1
buy: 20180428-21-05 49 12 28.9
buy: 20190304-01-07 58 45 24.8
buy: 20190111-23-08 78 9 28.8
buy: 20170704-07-11 81 2 25.5
buy: 20190320-18-02 85 0 13.5
☆hit!☆: 20190320-18-02 0 13.5
buy: 20180316-22-08 106 1 24.9
buy: 20180606-16-05 110 12 27.8
buy: 20180508-20-09 119 8 61.1
buy: 20180705-24-01 121 16 13.8
buy: 20180705-24-01 121 18 34.4
☆hit!☆: 20180705-24-01 18 34.4
buy: 20170521-23-08 129 25 36.5
buy: 20180319-12-05 139 0 22.0
buy: 20190117-01-06 147 1 29.6
buy: 20190117-01-06 147 8 31.2
buy: 20190117-01-06 147 16 30.9
buy: 201801

buy: 20180320-01-06 1393 0 13.3
buy: 20190211-21-09 1400 0 13.5
buy: 20180921-11-06 1401 0 51.7
buy: 20180921-11-06 1401 4 25.6
buy: 20180506-23-04 1414 8 21.9
buy: 20170421-08-03 1415 105 39.8
buy: 20180909-22-03 1421 0 37.4
buy: 20180909-22-03 1421 20 26.6
buy: 20170904-23-08 1426 0 26.9
buy: 20180201-02-02 1429 70 30.0
buy: 20170709-06-12 1431 4 16.5
buy: 20190222-03-11 1452 0 12.2
buy: 20170808-24-03 1455 23 36.7
buy: 20171231-20-09 1493 43 22.3
buy: 20190220-14-03 1497 2 24.2
buy: 20190220-14-03 1497 13 38.7
buy: 20180820-24-01 1499 7 16.8
buy: 20190211-21-06 1507 0 31.1
buy: 20171028-08-08 1513 6 34.3
buy: 20171028-08-08 1513 12 46.1
buy: 20180321-08-08 1516 17 27.2
buy: 20171208-18-03 1517 4 30.1
buy: 20171208-18-03 1517 5 27.9
buy: 20171208-18-03 1517 6 28.1
buy: 20180325-21-03 1519 1 27.0
buy: 20180317-24-04 1525 16 25.8
buy: 20180108-24-01 1560 16 23.8
buy: 20180108-24-01 1560 17 21.5
buy: 20180108-24-01 1560 18 15.6
buy: 20180311-14-01 1568 6 26.0
buy: 20170521-24-01 1574 8 

buy: 20180205-24-03 2786 4 37.0
buy: 20180206-12-04 2807 9 36.6
buy: 20180117-23-08 2809 0 30.8
buy: 20190429-12-07 2814 21 31.7
buy: 20180517-09-12 2815 8 18.2
buy: 20180219-06-07 2820 4 29.2
buy: 20171218-18-04 2821 3 15.9
buy: 20180522-17-12 2823 4 25.3
buy: 20170623-05-08 2838 0 29.3
buy: 20181020-13-02 2848 58 29.6
buy: 20181008-05-09 2855 0 24.0
buy: 20190114-11-05 2865 1 40.0
buy: 20190114-11-05 2865 4 12.7
buy: 20180802-22-03 2867 0 37.2
☆hit!☆: 20180802-22-03 0 37.2
buy: 20180802-22-03 2867 2 38.2
buy: 20180625-20-06 2871 4 22.3
buy: 20180625-20-06 2871 5 29.6
buy: 20190303-07-05 2872 0 31.0
buy: 20190303-07-05 2872 4 15.0
☆hit!☆: 20190303-07-05 4 15.0
buy: 20190405-21-06 2875 21 36.3
buy: 20171102-24-09 2890 9 26.4
buy: 20170416-17-05 2894 17 44.5
buy: 20181216-08-07 2895 1 33.3
buy: 20171222-23-01 2904 2 19.6
buy: 20171222-23-01 2904 5 33.1
buy: 20180101-08-06 2908 12 26.6
buy: 20180628-09-04 2942 33 31.6
buy: 20190101-23-03 2945 4 22.5
buy: 20190101-23-03 2945 5 19.6
buy: 2

buy: 20180612-12-03 4161 9 28.4
buy: 20170529-18-05 4167 9 35.1
buy: 20181202-09-12 4173 12 26.3
buy: 20170813-09-10 4204 4 26.9
buy: 20190429-07-08 4205 12 55.2
buy: 20181001-17-07 4208 0 22.2
buy: 20181130-13-01 4216 1 32.2
buy: 20171226-22-11 4220 0 30.0
buy: 20180226-10-01 4251 7 34.0
buy: 20190310-12-05 4276 1 14.6
buy: 20170612-17-07 4277 3 24.7
buy: 20170428-23-08 4278 4 36.9
buy: 20170428-23-08 4278 5 35.9
buy: 20170428-23-08 4278 6 34.0
buy: 20170428-23-08 4278 45 28.2
buy: 20171225-18-02 4318 0 77.2
buy: 20170723-05-08 4337 37 32.7
buy: 20190404-13-05 4340 0 20.3
buy: 20190404-13-05 4340 4 26.2
buy: 20180906-11-04 4345 1 25.1
buy: 20181210-21-08 4347 0 40.3
buy: 20181210-21-08 4347 16 49.2
buy: 20180924-18-01 4360 0 17.4
buy: 20180924-18-01 4360 2 22.4
buy: 20171129-01-01 4361 70 50.8
buy: 20180715-20-08 4367 0 23.1
☆hit!☆: 20180715-20-08 0 23.1
buy: 20170618-24-06 4369 6 27.3
buy: 20181007-10-08 4376 0 18.5
buy: 20180429-08-12 4377 0 31.3
buy: 20180429-08-12 4377 4 41.3
buy:

buy: 20180705-20-04 5561 4 22.6
buy: 20171015-06-05 5570 0 25.7
buy: 20171027-18-09 5575 0 29.2
buy: 20171027-18-09 5575 4 34.2
buy: 20170619-09-09 5582 5 31.5
buy: 20180920-12-08 5605 1 17.3
buy: 20180421-12-02 5606 1 24.2
buy: 20180421-12-02 5606 8 25.1
buy: 20181215-19-08 5625 0 14.3
buy: 20180921-15-02 5627 0 28.0
buy: 20180921-15-02 5627 1 16.9
buy: 20180921-15-02 5627 8 20.7
buy: 20170623-23-09 5631 12 39.4
buy: 20170914-23-01 5637 11 45.1
buy: 20180627-06-07 5641 4 30.1
buy: 20190327-22-08 5646 4 23.5
buy: 20171025-01-01 5647 25 37.1
buy: 20170416-02-05 5658 5 20.1
buy: 20190329-04-11 5673 19 47.3
buy: 20180609-17-04 5676 9 26.9
buy: 20180502-19-04 5681 16 29.3
buy: 20180813-20-08 5683 0 29.4
buy: 20180813-20-08 5683 4 14.6
buy: 20181229-02-02 5688 66 37.4
buy: 20171225-18-03 5693 1 91.7
buy: 20170506-08-08 5694 4 18.8
buy: 20170506-08-08 5694 5 22.3
buy: 20180112-13-06 5701 29 39.4
buy: 20180112-13-06 5701 77 32.6
☆hit!☆: 20180112-13-06 77 32.6
buy: 20181217-06-04 5710 5 19.3
b

...

buy: 20190303-11-03 21823 4 29.5
buy: 20190303-11-03 21823 6 38.6
buy: 20190303-10-05 21831 25 60.2
buy: 20190314-24-05 21843 9 16.4
buy: 20180513-12-08 21855 0 34.5
buy: 20180506-01-08 21861 12 20.2
buy: 20180506-01-08 21861 13 53.0
buy: 20180506-01-08 21861 17 127.6
buy: 20190115-10-08 21872 5 21.7
buy: 20170920-19-11 21876 0 29.2
buy: 20180903-08-07 21882 8 33.1
buy: 20180827-09-04 21897 2 36.0
buy: 20170918-19-05 21926 4 19.4
buy: 20171125-06-06 21938 1 35.6
buy: 20171125-06-06 21938 8 28.7
buy: 20180806-09-01 21939 0 26.8
buy: 20180806-09-01 21939 1 22.7
buy: 20180806-09-01 21939 2 53.9
buy: 20180806-09-01 21939 4 23.8
buy: 20170624-23-08 21945 12 25.8
buy: 20171009-05-02 21948 0 34.4
buy: 20171009-05-02 21948 20 22.5
buy: 20171002-20-04 21949 4 16.7
buy: 20171002-20-04 21949 9 31.3
buy: 20180412-10-01 21956 12 31.0
buy: 20170403-12-09 21957 0 28.0
buy: 20170403-12-09 21957 21 19.3
buy: 20180410-03-08 21961 4 22.2
buy: 20180410-03-08 21961 17 17.1
buy: 20180807-17-09 21965 9 20.3


buy: 20190122-24-05 23253 12 15.7
buy: 20170730-01-02 23260 0 40.2
buy: 20190107-19-06 23264 37 27.2
buy: 20181129-24-02 23287 17 37.8
buy: 20180428-13-04 23290 22 49.1
buy: 20190119-16-06 23300 8 32.0
buy: 20171215-04-05 23314 2 17.5
buy: 20180219-06-08 23329 6 18.4
buy: 20180219-06-08 23329 12 45.0
buy: 20180219-06-08 23329 80 44.8
buy: 20170812-06-05 23330 12 30.7
buy: 20170610-21-08 23332 0 41.1
buy: 20170610-21-08 23332 2 14.6
buy: 20181029-19-09 23336 4 26.5
buy: 20180216-23-08 23344 22 28.8
buy: 20180801-17-04 23346 8 19.0
buy: 20180527-09-05 23347 8 17.7
buy: 20171023-05-03 23349 26 30.4
buy: 20170807-24-05 23350 3 26.3
buy: 20170614-03-04 23352 38 37.2
buy: 20170612-24-09 23357 0 46.6
buy: 20170612-24-09 23357 4 34.1
buy: 20180205-05-01 23388 0 29.8
buy: 20180710-15-08 23406 10 21.7
buy: 20181213-02-02 23416 1 40.4
buy: 20181213-02-02 23416 8 24.3
buy: 20181213-02-02 23416 9 57.7
buy: 20180516-10-04 23433 8 26.5
buy: 20190115-14-05 23455 6 23.9
buy: 20170809-05-09 23464 2 32.3

buy: 20180702-05-09 24717 2 13.5
buy: 20180609-21-02 24719 5 18.9
☆hit!☆: 20180609-21-02 5 18.9
buy: 20190204-23-04 24721 17 24.9
buy: 20170728-20-06 24743 96 24.8
buy: 20170701-17-10 24748 0 14.8
buy: 20180324-22-07 24751 5 36.1
buy: 20180619-24-08 24753 2 28.6
buy: 20171029-18-04 24759 8 17.6
buy: 20170513-21-01 24762 1 38.8
buy: 20181215-02-07 24763 0 25.8
buy: 20180404-04-05 24781 0 28.1
buy: 20180404-04-05 24781 7 10.8
buy: 20170526-08-05 24782 37 22.5
buy: 20190303-07-12 24786 3 25.3
buy: 20180501-20-12 24804 0 12.1
buy: 20180812-22-01 24807 1 31.1
buy: 20170918-22-05 24812 3 31.3
buy: 20170918-22-05 24812 16 23.2
buy: 20181220-09-09 24825 9 31.6
buy: 20181002-24-02 24830 25 27.8
buy: 20190113-10-03 24834 0 21.6
buy: 20180319-13-05 24839 0 25.6
buy: 20190416-19-11 24849 100 41.9
buy: 20180709-11-06 24855 0 16.7
buy: 20190322-03-03 24856 17 22.8
☆hit!☆: 20190322-03-03 17 22.8
buy: 20181110-05-03 24858 0 40.6
buy: 20181110-05-03 24858 13 50.4
buy: 20190425-08-06 24869 4 44.6
buy: 2

buy: 20190103-06-03 26177 41 42.9
buy: 20180616-03-09 26180 0 47.1
buy: 20180616-03-09 26180 4 30.2
buy: 20171017-02-01 26190 37 34.1
buy: 20170807-23-03 26208 5 52.2
buy: 20181118-10-09 26210 1 26.9
buy: 20180816-06-11 26259 7 63.5
buy: 20170506-06-11 26285 0 33.2
buy: 20180510-18-12 26287 5 22.9
buy: 20171101-14-02 26308 70 18.9
buy: 20180820-19-01 26311 0 13.9
buy: 20180820-19-01 26311 3 30.6
buy: 20180417-21-01 26315 14 37.1
buy: 20180118-24-12 26318 16 31.3
buy: 20190309-18-09 26320 5 20.2
buy: 20180105-12-06 26324 9 13.4
buy: 20170411-02-06 26328 29 30.3
buy: 20170410-23-02 26336 0 15.1
buy: 20170410-23-02 26336 2 28.3
buy: 20190311-24-04 26349 76 44.2
buy: 20171025-16-02 26368 12 23.5
buy: 20171025-16-02 26368 13 30.1
buy: 20171025-16-02 26368 80 46.2
buy: 20170913-23-07 26380 1 24.9
buy: 20190319-10-03 26386 0 29.7
buy: 20181211-08-07 26405 0 19.6
buy: 20170404-13-07 26412 27 66.3
buy: 20190316-20-06 26423 4 31.1
buy: 20180201-16-10 26429 0 15.9
buy: 20190402-07-12 26443 9 39.6

buy: 20181125-17-09 27622 10 33.0
buy: 20170912-16-01 27635 16 23.6
buy: 20181116-06-05 27637 0 39.4
buy: 20181013-17-02 27640 4 31.7
buy: 20181029-13-05 27644 0 43.2
buy: 20181029-13-05 27644 4 22.5
buy: 20180722-07-07 27645 4 36.3
buy: 20190408-01-01 27662 5 49.0
buy: 20171115-10-03 27668 8 30.5
buy: 20180813-05-11 27685 0 28.9
buy: 20190428-05-01 27687 0 42.3
buy: 20190417-23-03 27688 0 29.0
buy: 20181022-01-02 27689 9 58.7
buy: 20180403-21-03 27702 5 101.5
buy: 20180428-13-08 27714 100 27.3
buy: 20171203-09-11 27734 9 28.0
buy: 20190201-21-10 27749 2 12.4
buy: 20180619-14-05 27759 0 39.2
buy: 20180619-14-05 27759 2 47.8
buy: 20180619-14-05 27759 4 18.8
buy: 20181102-10-07 27775 26 28.0
buy: 20181102-10-07 27775 46 19.8
buy: 20171224-17-01 27785 9 20.7
buy: 20171210-14-03 27786 101 28.9
buy: 20180808-21-05 27813 0 31.0
buy: 20180808-21-05 27813 1 25.8
buy: 20180808-21-05 27813 20 24.4
buy: 20180924-14-08 27824 37 55.5
buy: 20180531-13-03 27838 2 26.3
buy: 20180531-13-03 27838 22 28.

In [38]:
# trainの回収率を計算
y_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

#print(X_train.head())
#print(y_train[0:5])
#print(y_pred_max[0:5])
#c = collections.Counter(y_pred_max)
#print(len(c) )

# 精度 (Accuracy) を計算する
# エラーになるけど気にしない。
accuracy = sum(y_train == y_pred_max) / len(y_train)
print("accuracy:",accuracy)

# 回収率を計算
res=0
for i in range(len(y_train)):
    if y_train.iloc[i]==y_pred_max[i]:
        res += o_train[i] -1
    else:
        pass
print("return:",res/len(y_train))


ValueError: Unable to coerce to Series, length must be 1: given 84821

In [ ]:
print(y_pred[0])

In [ ]:
print(df.iloc[X_train.index]['raceId'])

In [ ]:
print(model.feature_importance())